In [1]:
from datasets import load_dataset
raw_datasets = load_dataset("code_search_net", "python")

/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [3]:
raw_datasets['train']

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [4]:
raw_datasets['train'].features

{'repository_name': Value(dtype='string', id=None),
 'func_path_in_repository': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'whole_func_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'func_code_string': Value(dtype='string', id=None),
 'func_code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'func_documentation_string': Value(dtype='string', id=None),
 'func_documentation_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'split_name': Value(dtype='string', id=None),
 'func_code_url': Value(dtype='string', id=None)}

In [5]:
print(raw_datasets['train']['whole_func_string'][1])

def initializable(self):
        """True if the Slot is initializable."""
        return bool(lib.EnvSlotInitableP(self._env, self._cls, self._name))


In [6]:
training_corpus = (
    raw_datasets['train'][i:i+1000]['whole_func_string'] for i in range(0, len(raw_datasets['train']), 1000)
)

In [7]:
type(training_corpus)

generator

In [8]:
def get_training_corpus():
    return (
        raw_datasets['train'][i:i+1000]['whole_func_string'] for i in range(0, len(raw_datasets['train']), 1000)
    )

training_corpus = get_training_corpus()

In [9]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained('gpt2')


/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [11]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [12]:
tokens = tokenizer.tokenize(example)
print(f'example : {example}')
print(f'tokens: {tokens}')

example : def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b
tokens: ['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [13]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

In [14]:
tokenizer.save_pretrained("code_search_net_tokenizer")

('code_search_net_tokenizer/tokenizer_config.json',
 'code_search_net_tokenizer/special_tokens_map.json',
 'code_search_net_tokenizer/vocab.json',
 'code_search_net_tokenizer/merges.txt',
 'code_search_net_tokenizer/added_tokens.json',
 'code_search_net_tokenizer/tokenizer.json')

In [15]:
from huggingface_hub import notebook_login,login
notebook_login()

In [16]:
tokenizer.push_to_hub("code_search_net_tokenizer")

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/NormanThink/code_search_net_tokenizer/commit/289825e16c9f8eb1c76be2ea8cf394956dd066df', commit_message='Upload tokenizer', commit_description='', oid='289825e16c9f8eb1c76be2ea8cf394956dd066df', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NormanThink/code_search_net_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='NormanThink/code_search_net_tokenizer'), pr_revision=None, pr_num=None)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("NormanThink/code_search_net_tokenizer")

FAST tokenizer

In [46]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example, return_offsets_mapping=True)


In [47]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [48]:
type(encoding['input_ids'])

list

In [49]:
type(tokenizer(example, return_tensors='pt')['input_ids'])

torch.Tensor

In [50]:
tokenizer.is_fast

True

In [51]:
encoding.is_fast

True

In [52]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

In [53]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

In [54]:
example

'My name is Sylvain and I work at Hugging Face in Brooklyn.'

In [55]:
encoding.word_to_tokens(0)


TokenSpan(start=1, end=2)

In [56]:
start,end = encoding.word_to_tokens(3)
example[start:end]

'ame '

In [57]:
offsets = encoding.offset_mapping

In [59]:
start,end

(4, 8)

In [62]:
offsets[start], offsets[end-1]

((11, 12), (16, 18))

In [63]:
print(offsets)

[(0, 0), (0, 2), (3, 7), (8, 10), (11, 12), (12, 14), (14, 16), (16, 18), (19, 22), (23, 24), (25, 29), (30, 32), (33, 35), (35, 40), (41, 45), (46, 48), (49, 57), (57, 58), (0, 0)]


In [44]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Input text
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."

# Tokenize input
encoding = tokenizer(example, return_offsets_mapping=True)

# Get token indices for word 3 ("Sylvain")
start_token, end_token = encoding.word_to_tokens(3)

# Extract character offsets from encoding
offsets = encoding.offset_mapping

# Character start and end positions for the word
char_start = offsets[start_token][0]
char_end = offsets[end_token - 1][1]

# Extract word from original example text
word = example[char_start:char_end]
print(word)


Sylvain


/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [64]:
from transformers import pipeline
token_classifier = pipeline('token-classification', model='dbmdz/bert-large-cased-finetuned-conll03-english')
token_classifier(example)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default

[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.99590707,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932106,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [66]:
token_classifier = pipeline('token-classification', model='dbmdz/bert-large-cased-finetuned-conll03-english', aggregation_strategy='simple')
token_classifier(example)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default

[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [67]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
model_checkpoint = 'dbmdz/bert-large-cased-finetuned-conll03-english'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
inputs = tokenizer(example, return_tensors='pt')
outputs = model(**inputs)

/home/osboxes/anaconda3/envs/huggingface/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exac

In [68]:
inputs['input_ids'].shape, outputs.logits.shape

(torch.Size([1, 19]), torch.Size([1, 19, 9]))

In [69]:
import torch
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]


In [70]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}